In [3]:
# Bước 1: Cài đặt các thư viện cần thiết (nếu chưa có)
# !pip install -q transformers datasets torch accelerate pandas

import pandas as pd
from datasets import Dataset, DatasetDict
from transformers import (
    GPT2LMHeadModel,
    GPT2Tokenizer,
    Trainer,
    TrainingArguments,
    DataCollatorForLanguageModeling,
    AutoTokenizer,
    pipeline
)
import torch


2025-05-02 13:09:25.072290: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1746191365.255486      31 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1746191365.312081      31 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [2]:
# -------- Bước 2: Tải và Chuẩn bị Dữ liệu --------

# Đường dẫn tới file dữ liệu của bạn
file_path = '/kaggle/input/mt-dataset/dataset.csv'
output_dir = './gpt2-finetuned-en-vi' # Thư mục để lưu mô hình
num_samples = 100000 # Số lượng dòng dữ liệu muốn sử dụng
val_split_ratio = 0.1 # Tỷ lệ dữ liệu dùng cho tập validation (10%)

try:
    # Đọc dữ liệu từ file CSV và chỉ lấy 'num_samples' dòng đầu tiên
    print(f"Đang đọc {num_samples} dòng đầu tiên từ {file_path}...")
    df = pd.read_csv(file_path).head(num_samples)

    # Kiểm tra xem cột 'en' và 'vi' có tồn tại không
    if 'en' not in df.columns or 'vi' not in df.columns:
        raise ValueError("File CSV phải chứa cột 'en' và 'vi'.")

    # Loại bỏ các hàng có giá trị thiếu trong cột 'en' hoặc 'vi'
    df.dropna(subset=['en', 'vi'], inplace=True)
    print(f"Số lượng mẫu sau khi loại bỏ giá trị thiếu: {len(df)}")

    # Định dạng lại dữ liệu
    formatted_texts = [f"English: {en}\nVietnamese: {vi}<|endoftext|>" for en, vi in zip(df['en'], df['vi'])]

    # Tạo đối tượng Dataset của Hugging Face
    data_dict = {'text': formatted_texts}
    full_dataset = Dataset.from_dict(data_dict)

    # Chia dữ liệu thành tập train và validation
    print(f"Đang chia dữ liệu thành tập train ({1-val_split_ratio:.0%}) và validation ({val_split_ratio:.0%})...")
    train_val_split = full_dataset.train_test_split(test_size=val_split_ratio, seed=42) # seed để đảm bảo kết quả chia nhất quán

    # Tạo DatasetDict để quản lý dễ dàng hơn
    split_datasets = DatasetDict({
        'train': train_val_split['train'],
        'validation': train_val_split['test']
    })

    print(f"Kích thước tập train: {len(split_datasets['train'])}")
    print(f"Kích thước tập validation: {len(split_datasets['validation'])}")
    print("Ví dụ một mẫu dữ liệu trong tập train:")
    print(split_datasets['train'][0]['text'])

except FileNotFoundError:
    print(f"Lỗi: Không tìm thấy file tại đường dẫn '{file_path}'. Vui lòng kiểm tra lại.")
    exit()
except ValueError as ve:
    print(f"Lỗi dữ liệu: {ve}")
    exit()
except Exception as e:
    print(f"Đã xảy ra lỗi không mong muốn khi xử lý dữ liệu: {e}")
    exit()

Đang đọc 100000 dòng đầu tiên từ /kaggle/input/mt-dataset/dataset.csv...
Số lượng mẫu sau khi loại bỏ giá trị thiếu: 99294
Đang chia dữ liệu thành tập train (90%) và validation (10%)...
Kích thước tập train: 89364
Kích thước tập validation: 9930
Ví dụ một mẫu dữ liệu trong tập train:
English: Thank you. (Applause)
Vietnamese: Cảm ơn mọi người. (Vỗ tay)<|endoftext|>


In [4]:
# -------- Bước 3: Tải Mô hình và Tokenizer --------

model_name = 'gpt2'
print(f"\nĐang tải tokenizer cho mô hình '{model_name}'...")
tokenizer = GPT2Tokenizer.from_pretrained(model_name, pad_token='<|pad|>')

print(f"Đang tải mô hình '{model_name}'...")
model = GPT2LMHeadModel.from_pretrained(model_name)
model.resize_token_embeddings(len(tokenizer))


Đang tải tokenizer cho mô hình 'gpt2'...


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

...

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

The new embeddings will be initialized from a multivariate normal distribution that has old embeddings' mean and covariance. As described in this article: https://nlp.stanford.edu/~johnhew/vocab-expansion.html. To disable this, use `mean_resizing=False`


Embedding(50258, 768)

In [4]:
# -------- Bước 4: Tokenize Dữ liệu --------

print("\nĐang tokenize dữ liệu cho cả tập train và validation...")

def tokenize_function(examples):
    tokenized_output = tokenizer(
        examples['text'],
        truncation=True,
        max_length=512,
        # Không cần padding ở đây, DataCollator sẽ xử lý
    )
    return tokenized_output

# Áp dụng hàm tokenize cho cả hai tập dữ liệu
tokenized_datasets = split_datasets.map(
    tokenize_function,
    batched=True,
    remove_columns=['text'] # Xóa cột text gốc
)

print("Tokenize dữ liệu hoàn tất.")
print("Thông tin tập dữ liệu đã tokenize:")
print(tokenized_datasets)



Đang tokenize dữ liệu cho cả tập train và validation...


Map:   0%|          | 0/89364 [00:00<?, ? examples/s]

Map:   0%|          | 0/9930 [00:00<?, ? examples/s]

Tokenize dữ liệu hoàn tất.
Thông tin tập dữ liệu đã tokenize:
DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask'],
        num_rows: 89364
    })
    validation: Dataset({
        features: ['input_ids', 'attention_mask'],
        num_rows: 9930
    })
})


In [5]:
# -------- Bước 5: Cấu hình Huấn luyện --------

print("\nĐang cấu hình các tham số huấn luyện...")

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False,
)

# Cập nhật TrainingArguments để bao gồm đánh giá
training_args = TrainingArguments(
    output_dir=output_dir,
    num_train_epochs=5, # Giữ nguyên hoặc tăng nếu cần
    per_device_train_batch_size=4, # Điều chỉnh nếu cần
    per_device_eval_batch_size=8, # Batch size cho đánh giá, thường có thể lớn hơn train batch size
    gradient_accumulation_steps=8, # Điều chỉnh nếu cần
    learning_rate=5e-5,
    weight_decay=0.01,
    warmup_steps=50,
    logging_dir='./logs',
    logging_steps=100,
    save_steps=500, # Nên khớp với eval_steps nếu load_best_model_at_end=True
    save_total_limit=2,
    fp16=torch.cuda.is_available(),
    eval_strategy="steps",       # Bật đánh giá theo số bước
    eval_steps=500,                    # Đánh giá sau mỗi 500 bước
    load_best_model_at_end=True,       # Tải mô hình tốt nhất (dựa trên eval_loss) vào cuối quá trình
    metric_for_best_model="loss",      # Sử dụng loss làm tiêu chí chọn mô hình tốt nhất
    greater_is_better=False,           # Loss càng nhỏ càng tốt
    report_to="none",
)


Đang cấu hình các tham số huấn luyện...


In [6]:
# -------- Bước 6: Huấn luyện Mô hình --------

print("\nChuẩn bị khởi tạo Trainer với tập train và validation...")

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets['train'],   # Sử dụng tập train đã tokenize
    eval_dataset=tokenized_datasets['validation'], # Sử dụng tập validation đã tokenize
    data_collator=data_collator,
    tokenizer=tokenizer,
)

print("Bắt đầu quá trình fine-tuning (bao gồm cả đánh giá)...")
# trainer.train()
print("Huấn luyện hoàn tất!")

# Đánh giá cuối cùng sau khi huấn luyện xong (Trainer sẽ tự làm nếu load_best_model_at_end=True)
# eval_results = trainer.evaluate()
print(f"Kết quả đánh giá cuối cùng trên tập validation: {eval_results}")


Chuẩn bị khởi tạo Trainer với tập train và validation...


/tmp/ipykernel_19/747178024.py:5: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Bắt đầu quá trình fine-tuning (bao gồm cả đánh giá)...


`loss_type=None` was set in the config but it is unrecognised.Using the default loss: `ForCausalLMLoss`.


Step,Training Loss,Validation Loss
500,0.937700,1.783491
1000,0.867900,1.639366
1500,0.797300,1.551577
2000,0.773500,1.490165
2500,0.767400,1.444815
3000,0.723400,1.414669
3500,0.721800,1.387002
4000,0.699500,1.365651
4500,0.692600,1.349573
5000,0.692700,1.338454


/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked t

Huấn luyện hoàn tất!


/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


Kết quả đánh giá cuối cùng trên tập validation: {'eval_loss': 1.3176332712173462, 'eval_runtime': 188.078, 'eval_samples_per_second': 52.797, 'eval_steps_per_second': 3.302, 'epoch': 4.996866887476502}


In [7]:
# -------- Bước 7: Lưu Mô hình --------
# Nếu load_best_model_at_end=True, mô hình tốt nhất đã được tải lại
# Nếu không, mô hình cuối cùng sẽ được lưu

print(f"\nĐang lưu mô hình tốt nhất (hoặc cuối cùng) và tokenizer vào thư mục '{output_dir}'...")
trainer.save_model(output_dir) # Lưu mô hình tốt nhất đã được tải lại (nếu load_best_model_at_end=True)
tokenizer.save_pretrained(output_dir)
print("Đã lưu mô hình và tokenizer.")


Đang lưu mô hình tốt nhất (hoặc cuối cùng) và tokenizer vào thư mục './gpt2-finetuned-en-vi'...
Đã lưu mô hình và tokenizer.

Thử nghiệm mô hình đã fine-tune:


Device set to use cuda:0
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


Prompt: English: This is a test.
Vietnamese:
Generated: English: This is a test.
Vietnamese: Đây là một danh dục.


# Thực nghiệm

In [7]:
import torch

output_dir = '/kaggle/input/weight-pretrained-gpt-mt/gpt2-finetuned-en-vi' # Thư mục để lưu mô hình

print("\nThử nghiệm mô hình đã fine-tune:")
generator = pipeline('text-generation', model=output_dir, tokenizer=output_dir, device=0 if torch.cuda.is_available() else -1)
prompt_en = "English: This is a test.\nVietnamese:"
generated_text = generator(prompt_en, max_length=60, num_return_sequences=1, pad_token_id=tokenizer.eos_token_id) # Thêm pad_token_id
print(f"Prompt: {prompt_en}")
print(f"Generated: {generated_text[0]['generated_text']}")


Thử nghiệm mô hình đã fine-tune:


Device set to use cuda:0
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


Prompt: English: This is a test.
Vietnamese:
Generated: English: This is a test.
Vietnamese: Đây là một thử trọng với điều gì.


# Evaluate

In [4]:
# -------- 1. Tải lại dữ liệu và lấy tập validation --------
import pandas as pd

file_path = '/kaggle/input/mt-dataset/dataset.csv' # Đường dẫn tới file gốc
num_samples = 100000
val_split_ratio = 0.1

try:
    print(f"Đang đọc {num_samples} dòng đầu tiên từ {file_path}...")
    df = pd.read_csv(file_path).head(num_samples)
    df.dropna(subset=['en', 'vi'], inplace=True)

    validation_data = Dataset.from_pandas(df).train_test_split(test_size=val_split_ratio, seed=42)['test']

    print(f"Đã tải {len(validation_data)} mẫu trong tập validation.")
    # Xem thử một mẫu
    print("Ví dụ mẫu validation:")
    print(validation_data[0])

except FileNotFoundError:
    print(f"Lỗi: Không tìm thấy file tại đường dẫn '{file_path}'.")
    exit()
except Exception as e:
    print(f"Đã xảy ra lỗi khi tải dữ liệu validation: {e}")
    exit()


Đang đọc 100000 dòng đầu tiên từ /kaggle/input/mt-dataset/dataset.csv...
Đã tải 9930 mẫu trong tập validation.
Ví dụ mẫu validation:
{'en': "It would come from the butcher's, it would come from presents.", 'vi': 'từ những người bán thịt hoặc trong các hộp quà.', '__index_level_0__': 59693}


In [5]:
# -------- 2. Tải mô hình, tokenizer và pipeline --------

# Đường dẫn đến thư mục chứa mô hình và tokenizer đã fine-tune
model_output_dir = '/kaggle/input/weight-pretrained-gpt-mt/gpt2-finetuned-en-vi'

print(f"\nĐang tải mô hình và tokenizer từ '{model_output_dir}'...")
try:
    # Tải tokenizer
    tokenizer = AutoTokenizer.from_pretrained(model_output_dir)
    # Đảm bảo pad_token được thiết lập đúng (quan trọng cho pipeline)
    if tokenizer.pad_token is None:
        tokenizer.pad_token = tokenizer.eos_token
        print("Cảnh báo: Tokenizer không có pad_token. Đã đặt pad_token = eos_token.")

    # Tải pipeline sinh văn bản
    # device=0 nếu có GPU, device=-1 cho CPU
    device = 0 if torch.cuda.is_available() else -1
    generator = pipeline(
        'text-generation',
        model=model_output_dir,
        tokenizer=tokenizer,
        device=device
    )
    print("Tải mô hình và pipeline hoàn tất.")
except Exception as e:
    print(f"Lỗi khi tải mô hình hoặc tokenizer: {e}")
    exit()



Đang tải mô hình và tokenizer từ '/kaggle/input/weight-pretrained-gpt-mt/gpt2-finetuned-en-vi'...


Device set to use cuda:0


Tải mô hình và pipeline hoàn tất.


In [12]:
# -------- 3. Viết hàm sinh câu tiếng Việt --------
import re

def generate_vietnamese(english_sentence, generator, tokenizer, max_length_multiplier=1.5, min_new_tokens=5):
    """
    Sinh câu tiếng Việt từ câu tiếng Anh sử dụng pipeline generator.

    Args:
        english_sentence (str): Câu tiếng Anh đầu vào.
        generator (transformers.Pipeline): Pipeline text-generation đã tải.
        tokenizer (transformers.PreTrainedTokenizer): Tokenizer tương ứng.
        max_length_multiplier (float): Hệ số nhân với độ dài prompt để xác định max_length.
        min_new_tokens (int): Số token mới tối thiểu cần sinh ra.

    Returns:
        str: Câu tiếng Việt được sinh ra (đã làm sạch).
    """
    # Tạo prompt theo định dạng đã huấn luyện
    prompt = f"English: {english_sentence}\nVietnamese:"

    # Ước tính độ dài tối đa hợp lý dựa trên prompt
    prompt_length = len(tokenizer.encode(prompt))
    max_length = int(prompt_length * max_length_multiplier) + min_new_tokens # Đảm bảo đủ dài
    # Giới hạn max_length tối đa để tránh quá dài (ví dụ: 100)
    max_length = min(max_length, prompt_length + 100) # Sinh tối đa 100 token mới

    try:
        # Sinh văn bản
        generated_outputs = generator(
            prompt,
            max_length=max_length,
            min_new_tokens=min_new_tokens, # Đảm bảo sinh ít nhất vài từ mới
            num_return_sequences=1,
            pad_token_id=tokenizer.pad_token_id, # Sử dụng pad_token_id đã định nghĩa
            eos_token_id=tokenizer.eos_token_id,
        )

        # Lấy phần văn bản được sinh ra
        full_generated_text = generated_outputs[0]['generated_text']

        # Trích xuất phần tiếng Việt sau prompt
        # Tìm vị trí kết thúc của prompt
        vietnamese_part = full_generated_text[len(prompt):].strip()

        # Loại bỏ token kết thúc chuỗi nếu có ở cuối
        if vietnamese_part.endswith(tokenizer.eos_token):
            vietnamese_part = vietnamese_part[:-len(tokenizer.eos_token)].strip()

        # Loại bỏ các token đặc biệt khác có thể bị sinh ra (ví dụ: nếu có <|pad|>)
        vietnamese_part = vietnamese_part.replace(tokenizer.pad_token, "").strip()
        
        # Đôi khi mô hình có thể sinh lại "English:" hoặc các phần không mong muốn
        # Cắt bỏ tại dấu xuống dòng đầu tiên nếu có
        vietnamese_part = re.split(r'\n', vietnamese_part)[0].strip()


        return vietnamese_part

    except Exception as e:
        print(f"Lỗi trong quá trình sinh câu cho: '{english_sentence}'. Lỗi: {e}")
        return "" # Trả về chuỗi rỗng nếu có lỗi

In [13]:
# -------- 4. Sinh dự đoán cho tập validation --------
from tqdm import tqdm

predictions = []
references = []

print(f"\nBắt đầu sinh dự đoán cho {len(validation_data)} mẫu validation...")
# Sử dụng tqdm để xem tiến trình
# Có thể giảm số lượng mẫu để test nhanh hơn: for sample in tqdm(validation_data.select(range(100))):
for sample in tqdm(validation_data):
    english_input = sample['en']
    reference_vietnamese = sample['vi']

    # Sinh câu dự đoán
    predicted_vietnamese = generate_vietnamese(english_input, generator, tokenizer)

    predictions.append(predicted_vietnamese)
    references.append(reference_vietnamese) # BLEU cần list các reference, nhưng ở đây mỗi sample chỉ có 1 ref

print("\nHoàn tất sinh dự đoán.")
print("Ví dụ một vài cặp dự đoán và tham chiếu:")
for i in range(min(5, len(predictions))):
    print(f"  EN: {validation_data[i]['en']}")
    print(f"  REF: {references[i]}")
    print(f"  PRED: {predictions[i]}")
    print("-" * 20)


Bắt đầu sinh dự đoán cho 9930 mẫu validation...


100%|██████████| 9930/9930 [26:08<00:00,  6.33it/s]


Hoàn tất sinh dự đoán.
Ví dụ một vài cặp dự đoán và tham chiếu:
  EN: It would come from the butcher's, it would come from presents.
  REF: từ những người bán thịt hoặc trong các hộp quà.
  PRED: Nó sẽ giữa t�
--------------------
  EN: So the only things I could manage to obtain was a kind of a compromise.
  REF: Vậy nên điều duy nhất mà tôi đã xoay sở để có được là một sự thoả hiệp tế nhị.
  PRED: Một điều mà tô
--------------------
  EN: Rather than me putting a dish down, they were allowed to help themselves to as much or as little as they wanted.
  REF: chứ không phải là theo ý của tôi. mọi người được phép gọi cho mình nhiều hay ít như họ muốn.
  PRED: Thay vì tôi lại bỏ sánh
--------------------
  EN: And everyone agrees that trees are beautiful, and I've never met anyone who says, "I don't like trees."
  REF: Và mọi người đều thống nhất cây cối đều đẹp. Vì tôi chưa từng nghe ai nói "Tôi chẳng thích cây" bao giờ,
  PRED: trên mọi người rằng
--------------------
  EN: In our time

In [18]:
# -------- 5. Tính điểm BLEU và ROUGE --------
!pip install -q evaluate sacrebleu rouge_score
import evaluate

print("\nĐang tính điểm BLEU và ROUGE...")

try:
    # Tải metric từ thư viện evaluate
    bleu_metric = evaluate.load("sacrebleu")
    rouge_metric = evaluate.load("rouge")

    # Tính điểm BLEU
    # Sacrebleu yêu cầu references là một list các list (vì một dự đoán có thể có nhiều tham chiếu)
    bleu_references = [[ref] for ref in references]
    bleu_results = bleu_metric.compute(predictions=predictions, references=bleu_references)
    print("\n--- Kết quả BLEU ---")
    print(f"BLEU Score: {bleu_results['score']:.4f}")

    # Tính điểm ROUGE
    # Rouge cần references là list các string
    rouge_results = rouge_metric.compute(predictions=predictions, references=references)
    print("\n--- Kết quả ROUGE ---")
    # In các điểm chính
    print(f"ROUGE-1: {rouge_results['rouge1']:.4f}")
    print(f"ROUGE-2: {rouge_results['rouge2']:.4f}")
    print(f"ROUGE-L: {rouge_results['rougeL']:.4f}")
    # print(f"ROUGE-Lsum (F1): {rouge_results['rougeLsum']:.4f}") # Thường dùng cho tóm tắt văn bản

except Exception as e:
    print(f"Lỗi khi tính toán điểm: {e}")

print("\nHoàn thành!")

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


  Preparing metadata (setup.py) ... done

Đang tính điểm BLEU và ROUGE...

--- Kết quả BLEU ---
BLEU Score: 0.7050

--- Kết quả ROUGE ---
ROUGE-1 (F1): 0.3847
ROUGE-2 (F1): 0.2342
ROUGE-L (F1): 0.3517

Hoàn thành!
